In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)


In [6]:
# View all of the classes that automap found
Base.classes.keys()

['station', 'measurement']

In [7]:
# Save references to each table
ST = Base.classes.station
MS = Base.classes.measurement

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
#session.query(func.count(ST.date)).all()
session.query(func.count(MS.date)).all()

[(19550,)]

In [10]:
# Find the most recent date in the data set.
session.query(MS.date).order_by(MS.date.desc()).first()

('2017-08-23',)

In [11]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
columnss = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])
for c in columnss:
    print(c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [19]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
date = dt.datetime(2017,8,23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
thession = session.query(MS.date).filter(MS.date > '2016-08-22').order_by(MS.date.desc())
meshin = thession.with_entities(MS.id,MS.date, MS.prcp).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(meshin,columns=['id','date', 'prcp'])
df
# Sort the dataframe by date
sorts = df.dropna(how='any')
sorts
sorts.set_index('date', inplace = True)
sorts
# Use Pandas Plotting with Matplotlib to plot the data


,id,prcp
date,,
2017-08-23,2724,0.00
2017-08-23,7635,0.00
2017-08-23,12187,0.08
2017-08-23,19550,0.45
2017-08-22,2723,0.00
...,...,...
2016-08-23,5091,0.15
2016-08-23,7331,0.05
2016-08-23,11851,0.02


In [13]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [14]:
# Design a query to calculate the total number of stations in the dataset


In [15]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.


In [16]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [17]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close Session

In [18]:
# Close Session
session.close()